In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import numpy


In [ ]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *

In [ ]:
import fastai; fastai.__version__

In [ ]:
np.random.seed(42)

In [ ]:
def noop(x): return x

class Flatten(nn.Module):
    def forward(self, x): return x.view(x.size(0), -1)

def conv(ni, nf, ks=3, stride=1, bias=False):
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

In [ ]:
act_fn = nn.ReLU(inplace=True)

def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)

def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
    bn = nn.BatchNorm2d(nf)
    nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
    layers = [conv(ni, nf, ks, stride=stride), bn]
    if act: layers.append(act_fn)
    return nn.Sequential(*layers)

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, expansion, ni, nh, stride=1):
        super().__init__()
        nf,ni = nh*expansion,ni*expansion
        layers  = [conv_layer(ni, nh, 3, stride=stride),
                   conv_layer(nh, nf, 3, zero_bn=True, act=False)
        ] if expansion == 1 else [
                   conv_layer(ni, nh, 1),
                   conv_layer(nh, nh, 3, stride=stride),
                   conv_layer(nh, nf, 1, zero_bn=True, act=False)
        ]
        self.convs = nn.Sequential(*layers)
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
        self.pool = noop if stride==1 else nn.AvgPool2d(2, ceil_mode=True)

    def forward(self, x): return act_fn(self.convs(x) + self.idconv(self.pool(x)))

In [ ]:
class XResNet(nn.Sequential):
    @classmethod
    def create(cls, expansion, layers, c_in=3, c_out=1000):
        nfs = [c_in, (c_in+1)*8, 64, 64]
        stem = [conv_layer(nfs[i], nfs[i+1], stride=2 if i==0 else 1)
            for i in range(3)]

        nfs = [64//expansion,64,128,256,512]
        res_layers = [cls._make_layer(expansion, nfs[i], nfs[i+1],
                                      n_blocks=l, stride=1 if i==0 else 2)
                  for i,l in enumerate(layers)]
        res = cls(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *res_layers,
            nn.AdaptiveAvgPool2d(1), Flatten(),
            nn.Linear(nfs[-1]*expansion, c_out),
        )
        init_cnn(res)
        return res

    @staticmethod
    def _make_layer(expansion, ni, nf, n_blocks, stride):
        return nn.Sequential(
            *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1)
              for i in range(n_blocks)])

In [ ]:
def xresnet18 (**kwargs): return XResNet.create(1, [2, 2,  2, 2], **kwargs)
def xresnet34 (**kwargs): return XResNet.create(1, [3, 4,  6, 3], **kwargs)
def xresnet50 (**kwargs): return XResNet.create(4, [3, 4,  6, 3], **kwargs)
def xresnet101(**kwargs): return XResNet.create(4, [3, 4, 23, 3], **kwargs)
def xresnet152(**kwargs): return XResNet.create(4, [3, 8, 36, 3], **kwargs)

In [ ]:
src = (ImageList.from_folder(path='train-snakes-cropped').split_by_rand_pct(0.2).label_from_folder())

In [ ]:
# src.add_test_folder('/home/ubuntu/snakes_breed/round1')

In [ ]:
epochs = 40
lr = 1e-2
bs = 24
mixup = 0.2
image_size = 512

In [ ]:
tfms=(rand_zoom(scale=(0.75,1.33)),[crop_pad()])

In [ ]:
# tfms = get_transforms(do_flip=True,flip_vert=False,max_rotate=10.0,max_zoom=1.1,max_lighting=0.2,max_warp=0.2,p_affine=0.75,p_lighting=0.75)


In [ ]:
stats_to_norm = [tensor([0.4762, 0.4438, 0.3767]), tensor([0.2230, 0.2163, 0.2121])]

In [ ]:
data = (src.transform(tfms, size=image_size)
        .databunch(bs=bs).normalize(stats_to_norm))

In [ ]:
data.stats

In [ ]:
data.show_batch(3,figsize=(15,9))

In [ ]:
opt_func = partial(optim.Adam, betas=(0.9,0.99), eps=1e-6)


In [ ]:
m = xresnet152(c_out=45)


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=1.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()
loss_func = FocalLoss(gamma=2.5)

In [ ]:
learn = (Learner(data, m, opt_func=opt_func,
             metrics=[accuracy,FBeta(beta=1,average='macro')],
             bn_wd=False, true_wd=True,
             loss_func = loss_func)
            )

In [ ]:
if mixup: learn = learn.mixup(alpha=mixup)


In [ ]:
from fastai.metrics import accuracy, top_k_accuracy

top_2_accuracy = partial(top_k_accuracy, k=2)

In [ ]:
learn.metrics = [accuracy,top_2_accuracy]

In [ ]:
learn.load('c_xresnet-512-round-3')

In [ ]:
learn = learn.to_fp16(dynamic=True)


In [ ]:
learn.lr_find()

In [ ]:
learn.metrics = [accuracy,FBeta(beta=1,average='macro'),top_2_accuracy]

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, 1e-5)

In [ ]:
learn.save('c_xresnet-rand-zoom-512-round-1')

In [ ]:
lr=1e-5

In [ ]:
learn.fit(10, lr)

In [ ]:
learn.save('c_xresnet-512-round-2')

In [ ]:
lr=1e-6

In [ ]:
learn.fit(10,lr)

In [ ]:
learn.save('c_xresnet-512-round-3')

In [ ]:
learn.loss_func = FocalLoss(gamma=2.)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit(2,lr)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=75,normalize=True)


In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=75,normalize=False)

In [ ]:
interp.plot_top_losses(6, figsize=(30,22))

In [ ]:
learn.save('c_xresnet-512-round-4')

In [ ]:
learn.loss_func = LabelSmoothingCrossEntropy()

In [ ]:
learn.fit(2,lr)

In [ ]:
learn.save('c_xresnet-512-round-5')

In [ ]:
lr=1e-5

In [ ]:
learn.fit(2,lr)

In [ ]:
learn.save('c_xresnet-512-round-6')

In [ ]:
learn